In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.special import hermite
from scipy.special import factorial
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import os
if not os.path.exists("loss_data"):
	os.makedirs("loss_data")
if not os.path.exists("model_weights"):
	os.makedirs("model_weights")
if not os.path.exists("results"):
	os.makedirs("results")
if not os.path.exists("plots"):
	os.makedirs("plots")

In [3]:
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=int(1000/fps), loop=loop)

def plot_result(x,y,x_data,y_data,yh,xlim_low,xlim_high,ylim_low,ylim_high,xp=None,energy=None,n=None):
    "Pretty plot training results"
    plt.figure(figsize=(8,4))
    plt.plot(x,y, color="grey", linewidth=2, alpha=0.8, label="Exact solution")
    plt.plot(x,yh, color="tab:blue", linewidth=3, alpha=0.8, label="Neural network prediction")
    plt.scatter(x_data, y_data, s=60, color="tab:orange", alpha=0.8, label='Training data')
    if xp is not None:
        plt.scatter(xp.detach().cpu().numpy(), (-0*torch.ones_like(xp)).detach().cpu().numpy(), s=60, color="tab:green", alpha=0.8,
                    label='Physics loss training locations')
    if energy is not None and n is not None:
        plt.text(15.51, 0.34, f"Predicted Energy: {energy:.2f}", fontsize="large", color="k")
        plt.text(15.51, 0.43, f"True Energy: {n + 0.5:.2f}", fontsize="large", color="k")
    l = plt.legend(loc=(1.01,0.34), frameon=False, fontsize="large")
    plt.setp(l.get_texts(), color="k")
    plt.xlim(xlim_low, xlim_high)
    plt.ylim(ylim_low, ylim_high)
    plt.text(1.065,0.7,"Training step: %i"%(i+1),fontsize="xx-large",color="k")
    plt.axis("off")

import matplotlib.animation as animation

# def save_mp4(outfile, files, fps=30):
#     """
#     Save a sequence of PNG files as an MP4 video.
#     """
#     fig = plt.figure()
#     plt.axis('off')  # optional: remove axis if just showing images

#     imgs = []
#     for fname in files:
#         img = Image.open(fname)
#         imgs.append([plt.imshow(img, animated=True)])

#     ani = animation.ArtistAnimation(fig, imgs, interval=1000/fps, blit=True)
#     ani.save(outfile, fps=fps, extra_args=['-vcodec', 'libx264'])
#     plt.close(fig)

def save_mp4(outfile, files, fps=30):
    """
    Save a sequence of PNG files as an MP4 video with minimal white padding.
    """
    # Open the first image to get its size
    img = Image.open(files[0])
    width, height = img.size

    # DPI = 100 gives figure size in inches as pixels/100
    dpi = 100
    figsize = (width / dpi, height / dpi)
    
    fig = plt.figure(figsize=figsize, dpi=dpi)
    plt.axis('off')

    imgs = []
    for fname in files:
        img = Image.open(fname)
        imgs.append([plt.imshow(img, animated=True)])

    ani = animation.ArtistAnimation(fig, imgs, interval=1000/fps, blit=True)
    ani.save(outfile, fps=fps, extra_args=['-vcodec', 'libx264'])
    plt.close(fig)


In [4]:

# def quantum_harmonic_oscillator(n, x):
#     """
#     Quantum harmonic oscillator state function for n-th energy level.

#     Parameters:
#     - n: Quantum number
#     - x: Position (torch.Tensor)

#     Returns:
#     - y: The n-th state wave function evaluated at x
#     """

#     m = 1
#     omega = 1
#     hbar = 1
#     prefactor = ((m*omega)/(np.pi*hbar))**0.25
#     normalization = 1 / np.sqrt(2**n * factorial(n))
#     x_np = x.numpy()

#     H_n = hermite(n)(np.sqrt(m*omega/hbar)*x_np)

#     y_np = prefactor * normalization * H_n * np.exp(-m*omega*x_np**2 / (2*hbar))

#     y = torch.from_numpy(y_np).type_as(x)
#     return y

# n = 10
# x = torch.linspace(-12,12,10000).view(-1,1)
# y = quantum_harmonic_oscillator(n, x).view(-1,1)
# print(x.shape, y.shape)

# x_start, x_end = -7, 0
# num_points = 40
# x_data = torch.linspace(x_start, x_end, num_points).view(-1, 1)
# y_data = quantum_harmonic_oscillator(n, x_data).view(-1, 1)
# print(x_data.shape, y_data.shape)

# plt.figure()
# plt.plot(x, y, label="Exact solution")
# plt.scatter(x_data, y_data, color="tab:orange", label="Training data")
# plt.legend()
# plt.show()

In [5]:

# class SineActivation(nn.Module):
#     def forward(self, x):
#         return torch.sin(x)

# class FCN(nn.Module):
#     def __init__(self, N_OUTPUT, N_HIDDEN, N_LAYERS):
#         super(FCN, self).__init__()
#         self.activation = SineActivation()
#         self.initial_layer = nn.Linear(1, N_HIDDEN)
#         self.hidden_layers = nn.ModuleList([nn.Linear(N_HIDDEN, N_HIDDEN) for _ in range(N_LAYERS - 1)])
#         self.output_layer = nn.Linear(N_HIDDEN, N_OUTPUT)

#     def forward(self, x):
#         x = self.activation(self.initial_layer(x))
#         for hidden_layer in self.hidden_layers:
#             x = self.activation(hidden_layer(x))
#         x = self.output_layer(x)
#         return x


# N_OUTPUT = 1
# N_HIDDEN = 50
# N_LAYERS = 3

# model = FCN(N_OUTPUT, N_HIDDEN, N_LAYERS)

In [6]:
# x_physics = torch.linspace(-12,12,50).view(-1,1).requires_grad_(True)

# torch.manual_seed(123)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# optimizer = torch.optim.Adam(model.parameters(),lr=5e-3)

# files = []
# data_loss_history = []
# physics_loss_history = []
# total_loss_history = []


# for i in range(1000):
#     optimizer.zero_grad()


#     yh = model(x_data)
#     loss1 = torch.mean((yh-y_data)**2)
#     data_loss_history.append(loss1.item())


#     yhp = model(x_physics)
#     dx  = torch.autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]
#     dx2 = torch.autograd.grad(dx,  x_physics, torch.ones_like(dx),  create_graph=True)[0]
#     physics = dx2 - (x_physics ** 2) * yhp + 2 * (n + 0.5) * yhp
#     loss2 = 5e-1 * torch.mean(physics ** 2)
#     physics_loss_history.append(loss2.item())
#     # loss3 = (torch.inner(yhp.squeeze(dim = 1).detach().cpu(), yhp.squeeze(dim = 1).detach().cpu()) - 1)**2
#     sym_yhp = model(-x_physics)
#     loss3 = torch.mean((yhp - sym_yhp)**2)


#     loss = loss1 + loss2 + loss3
#     total_loss_history.append(loss.item())
#     loss.backward()
#     optimizer.step()


#     if (i+1) % 5 == 0:
#         yh = model(x).detach()
#         xp = x_physics.detach()
#         xlim_low = -13
#         xlim_high = 13
#         ylim_low = -0.65
#         ylim_high = 0.65
#         plot_result(x,y,x_data,y_data,yh,xlim_low,xlim_high,ylim_low,ylim_high,xp=None)

#         file = "plots/qhm_pinn_10_%.8i.png"%(i+1)
#         plt.savefig(file, bbox_inches='tight', pad_inches=0.1, dpi=100, facecolor="white")
#         files.append(file)

#         if (i+1) % 800 == 0: plt.show()
#         else: plt.close("all")
# # save_gif_PIL("results/qhm_pinn_10_pinn_1.mp4", files, fps=30, loop=0)
# save_mp4("results/qhm_pinn_10_pinn_1.mp4", files, fps=30)

# Variation

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.special import hermite
from scipy.special import factorial
import matplotlib.pyplot as plt

def quantum_harmonic_oscillator(n, x):
    """
    Quantum harmonic oscillator state function for n-th energy level.

    Parameters:
    - n: Quantum number
    - x: Position (torch.Tensor)

    Returns:
    - y: The n-th state wave function evaluated at x
    """
    # Given values
    m = 1  # mass
    omega = 1  # angular frequency
    hbar = 1  # reduced Planck's constant
    prefactor = ((m*omega)/(np.pi*hbar))**0.25
    normalization = 1 / np.sqrt(2**n * factorial(n))
    x_np = x.numpy()
    # Calculate the Hermite polynomial H_n
    H_n = hermite(n)(np.sqrt(m*omega/hbar)*x_np)
    # Compute the wave function
    y_np = prefactor * normalization * H_n * np.exp(-m*omega*x_np**2 / (2*hbar))
    # Convert the result back to a torch.Tensor
    y = torch.from_numpy(y_np).type_as(x)
    return y

# Custom Sine activation function
class SineActivation(nn.Module):
    def forward(self, x):
        return torch.sin(x)

class FCN(nn.Module):
    def __init__(self, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super(FCN, self).__init__()
        self.activation = SineActivation()  # Use the sine activation function
        self.initial_layer = nn.Linear(1, N_HIDDEN)
        self.hidden_layers = nn.ModuleList([nn.Linear(N_HIDDEN, N_HIDDEN) for _ in range(N_LAYERS - 1)])
        self.output_layer = nn.Linear(N_HIDDEN, N_OUTPUT)
        self.energy = nn.Parameter(torch.tensor(1.0, requires_grad=True))

    def forward(self, x):
        x = self.activation(self.initial_layer(x))
        for hidden_layer in self.hidden_layers:
            x = self.activation(hidden_layer(x))
        x = self.output_layer(x)
        return x

# Example usage
N_OUTPUT = 1  # Output size
N_HIDDEN = 50  # Number of neurons in the hidden layers
N_LAYERS = 3  # Number of hidden layers

model = FCN(N_OUTPUT, N_HIDDEN, N_LAYERS)

# get the analytical solution over the full domain
n = 1 # nth eigenstate
x = torch.linspace(-12,12,10000).view(-1,1)
y = quantum_harmonic_oscillator(n, x).view(-1,1)

# Generate 20 equally spaced data points within the range [-7, 7]
x_start, x_end = -7, 7
num_points = 40
x_data = torch.linspace(x_start, x_end, num_points).view(-1, 1)
y_data = quantum_harmonic_oscillator(n, x_data).view(-1, 1)


x_physics = torch.linspace(-12,12,50).view(-1,1).requires_grad_(True) # sample locations over the problem domain

torch.manual_seed(123)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

optimizer = torch.optim.Adam(model.parameters(),lr=5e-3)

files = []

#plt.close()
for i in range(200):
    optimizer.zero_grad()

    # compute the "data loss"
    yh = model(x_data)
    loss1 = torch.mean((yh-y_data)**2)# use mean squared error# use mean squared error

    # compute the "physics loss"
    yhp = model(x_physics)
    dx  = torch.autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]# computes dy/dx
    dx2 = torch.autograd.grad(dx,  x_physics, torch.ones_like(dx),  create_graph=True)[0]# computes d^2y/dx^2
    # physics = dx2 - (x_physics ** 2) * yhp + 2 * (n + 0.5) * yhp
    physics = dx2 - (x_physics ** 2) * yhp + 2 * model.energy * yhp
    loss2 = 5e-3 * torch.mean(physics ** 2)

    # backpropagate joint loss
    loss = loss1 + loss2 # add two loss terms together
    loss.backward()
    optimizer.step()

    if (i+1) % 20 == 0:
        yh = model(x).detach()
        # xp = x_physics.detach()
        xlim_low = -13
        xlim_high = 13
        ylim_low = -0.65
        ylim_high = 0.65
        plot_result(x,y,x_data,y_data,yh,xlim_low,xlim_high,ylim_low,ylim_high,xp=x_physics,energy=model.energy.item(),n=n)

        file = "plots/qhm_pinn_10_%.8i.png"%(i+1)
        plt.savefig(file, bbox_inches='tight', pad_inches=0.1, dpi=100, facecolor="white")
        files.append(file)

        if (i+1) % 800 == 0: plt.show()
        else: plt.close("all")
# save_gif_PIL("results/qhm_pinn_10_pinn_1.mp4", files, fps=30, loop=0)
if x_end == 0:
    save_mp4(f"results/qhm_pinn_{n}_with_half_training_data_points.mp4", files, fps=30)
else:
    save_mp4(f"results/qhm_pinn_{n}_with_full_training_data_points.mp4", files, fps=30)

In [8]:
from torchviz import make_dot

model = FCN(N_OUTPUT=1, N_HIDDEN=32, N_LAYERS=4)
x = torch.randn(1, 1, requires_grad=True)
y = model(x)

make_dot(y, params=dict(model.named_parameters())).render("fcn_model", format="png")

'fcn_model.png'

In [10]:
torch.onnx.export(model, x, "fcn_model.onnx", input_names=['input'], output_names=['output'], opset_version=11)